In [1]:
from __future__ import print_function
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.model_selection import  train_test_split
import time #helper libraries
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from numpy import newaxis

import time
import intrinio_sdk
from intrinio_sdk.rest import ApiException
from pprint import pprint
import math
intrinio_sdk.ApiClient().configuration.api_key['api_key'] = 'OmNjN2M0YzRjMjNhZjQ1Zjc3NzZiODI4MmNjMjc4Njgw'

Using TensorFlow backend.


In [2]:
#security_api = intrinio_sdk.SecurityApi()
index_api = intrinio_sdk.IndexApi()

identifier = '$SPX' # str | A Security identifier (Ticker, FIGI, ISIN, CUSIP, Intrinio ID)
tag = 'level'
#type = ''
start_date = '2010-01-01' # date | Return prices on or after the date (optional)
end_date = '2020-08-04' # date | Return prices on or before the date (optional)sort_order = 'desc'
page_size = 3000 # int | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

api_response = index_api.get_stock_market_index_historical_data(identifier,tag, start_date=start_date, end_date=end_date, page_size=page_size, next_page=next_page)
#api_response

In [3]:
##############################################################################################################
df1 = pd.DataFrame(api_response.historical_data_dict)
df1.sort_values(["date"], axis=0, 
                 ascending=True, inplace=True) 
df1.set_index('date',inplace=True)
df1 = df1.dropna()
df1.tail()

,value
date,
2020-07-29,3258.44
2020-07-30,3246.22
2020-07-31,3271.12
2020-08-03,3294.61
2020-08-04,3306.51


In [4]:
df1.head()

,value
date,
2010-08-05,1125.81
2010-08-06,1121.64
2010-08-09,1127.79
2010-08-10,1121.06
2010-08-11,1089.47


In [5]:
len(df1)

2517

In [6]:
#Create a new dataframe with only the 'Close' column
data = df1.filter(['value'])
#Converting the dataframe to a numpy array
dataset = data.values
#Get /Compute the number of rows to train the model on
training_data_len = math.ceil( len(dataset) *.8) 

In [7]:
#Scale the all of the data to be values between 0 and 1 
scaler = MinMaxScaler(feature_range=(0, 1)) 
scaled_data = scaler.fit_transform(dataset)

In [8]:
#Create the scaled training data set 
train_data = scaled_data[0:training_data_len  , : ]
#Split the data into x_train and y_train data sets
x_train=[]
y_train = []
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

In [9]:
#Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [10]:
#Reshape the data into the shape accepted by the LSTM
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [11]:
#Build the LSTM network model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [12]:
#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
#Train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)


Epoch 1/1
1954/1954 [==============================] - 108s 55ms/step - loss: 0.0013


In [14]:
#Create a new dataframe
new_df = df1.filter(['value'])
#Get teh last 60 day closing price 
last_60_days = new_df[-60:].values
#Scale the data to be values between 0 and 1
last_60_days_scaled = scaler.transform(last_60_days)
#print(last_60_days_scaled)

In [15]:
#Create an empty list
X_test = []
#Append teh past 60 days
X_test.append(last_60_days_scaled)
#Convert the X_test data set to a numpy array
X_test = np.array(X_test)
#print(x_test)

In [16]:
#Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [17]:
#Get the predicted scaled price
pred_price = model.predict(X_test)
predictions = scaler.inverse_transform(pred_price)
print(predictions)

[[3290.5166]]
